In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from datetime import datetime
import re
from textblob import TextBlob
from nltk.translate.bleu_score import sentence_bleu

In [2]:
PATH_ROOT = "/home/shaul/workspace/GitHub/SOTA/"

In [3]:
cd {PATH_ROOT}

/home/shaul/workspace/GitHub/SOTA


In [4]:
for i, file in enumerate(glob.glob('data/raw_data/*.csv')):
    if i==0:
        df = pd.read_csv(file, index_col = 0)
        df['dataset'] = file[5:-4].lower()
    else:
        temp = pd.read_csv(file, index_col = 0)
        temp['dataset'] = file[5:-4].lower()
        df = pd.concat((df,temp),axis = 0)
df['random'] = df.dataset.apply(lambda x: 'random' in  x).astype(int)
df['duration'] = pd.to_datetime(df.SubmitTime)-pd.to_datetime(df.AcceptTime)


df = df.drop(['HITId','HITTypeId', 'Title', 'Description', 'Keywords',
       'RequesterAnnotation', 'AssignmentDurationInSeconds', 'AssignmentId','AssignmentStatus','AcceptTime', 'SubmitTime','AutoApprovalTime', 'ApprovalTime', 'RejectionTime','RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate','Last30DaysApprovalRate', 'Last7DaysApprovalRate','Approve', 'Reject'],axis = 1)

df.columns = ['annotator', 'text1','text2','label','dataset','random','duration']

df.label = df.label.apply(lambda x:x[0]).astype(int)

df = df.reset_index()
df['total_seconds'] = [int(df['duration'][i].total_seconds()) for i in range(df.shape[0])]
df.drop(columns="index",inplace=True)
df.to_csv('data/combined_dataset.csv')

In [5]:
df = pd.read_csv("data/combined_dataset.csv")

In [6]:
df.head(3)

,Unnamed: 0,annotator,text1,text2,label,dataset,random,duration,total_seconds
0,0,A2V7RTBYCIY1CC,"if he throw s you around , li ter ally , then ...",! we call them may day basket s,2,raw_data/gyafc_rewrites_random_human,1,0 days 00:06:45.000000000,405
1,1,A2GZNOA5CIVKUB,"if he throw s you around , li ter ally , then ...",! we call them may day basket s,3,raw_data/gyafc_rewrites_random_human,1,0 days 00:00:03.000000000,3
2,2,A3TKUXUTDX6FBF,"if he throw s you around , li ter ally , then ...",! we call them may day basket s,4,raw_data/gyafc_rewrites_random_human,1,0 days 00:00:11.000000000,11


Although I can just go straight to observe the difference between the (rudimentary approx. of) sentiment and the label score - it would be fair because the two sentences could also be semantically very different as well as be sentimentally diverging.

Therefore to better observe the phenomenon, lets highlight cases which have a very high word-overlap and see in those cases when the sentiment is different if they score high or low.

In [7]:
df['sentiment_dif'] = abs(df.text1.apply(lambda x: TextBlob(x).sentiment.polarity)-df.text2.apply(lambda x: TextBlob(x).sentiment.polarity))

In [8]:
df.head(3)

,Unnamed: 0,annotator,text1,text2,label,dataset,random,duration,total_seconds,sentiment_dif
0,0,A2V7RTBYCIY1CC,"if he throw s you around , li ter ally , then ...",! we call them may day basket s,2,raw_data/gyafc_rewrites_random_human,1,0 days 00:06:45.000000000,405,0.0
1,1,A2GZNOA5CIVKUB,"if he throw s you around , li ter ally , then ...",! we call them may day basket s,3,raw_data/gyafc_rewrites_random_human,1,0 days 00:00:03.000000000,3,0.0
2,2,A3TKUXUTDX6FBF,"if he throw s you around , li ter ally , then ...",! we call them may day basket s,4,raw_data/gyafc_rewrites_random_human,1,0 days 00:00:11.000000000,11,0.0


In [9]:
pairs = []
for index, row in df.iterrows():
    first_sentence_tokens = row['text1'].strip().split()
    second_sentence_tokens = row['text2'].strip().split()
    pairs.append((first_sentence_tokens, second_sentence_tokens))

In [10]:
scores_bleu = []
scores_bleu1 = []
for first_sentence_tokens, second_sentence_tokens in pairs:
    score_bleu = sentence_bleu([first_sentence_tokens], second_sentence_tokens)
    scores_bleu.append(score_bleu)

    score_bleu1 = sentence_bleu([first_sentence_tokens], second_sentence_tokens, weights=(1, 0, 0, 0))
    scores_bleu1.append(score_bleu1)


print(np.mean(scores_bleu))
print(np.std(scores_bleu))

print(np.mean(scores_bleu1))
print(np.std(scores_bleu1))

0.08892479824498775
0.19533510418693692
0.30581196288031903
0.2615081970765116


In [11]:
df['bleu_score'] = scores_bleu
df['bleu_score_1'] = scores_bleu1

In [13]:
df.head(5)

,Unnamed: 0,annotator,text1,text2,label,dataset,random,duration,total_seconds,sentiment_dif,bleu_score,bleu_score_1
0,0,A2V7RTBYCIY1CC,"if he throw s you around , li ter ally , then ...",! we call them may day basket s,2,raw_data/gyafc_rewrites_random_human,1,0 days 00:06:45.000000000,405,0.000,3.103614e-232,0.035813
1,1,A2GZNOA5CIVKUB,"if he throw s you around , li ter ally , then ...",! we call them may day basket s,3,raw_data/gyafc_rewrites_random_human,1,0 days 00:00:03.000000000,3,0.000,3.103614e-232,0.035813
2,2,A3TKUXUTDX6FBF,"if he throw s you around , li ter ally , then ...",! we call them may day basket s,4,raw_data/gyafc_rewrites_random_human,1,0 days 00:00:11.000000000,11,0.000,3.103614e-232,0.035813
3,3,A1SK0GV4SSOH1A,"of course , i have also seen brilliant women w...",":) anyways , you ask what love is blind really...",3,raw_data/gyafc_rewrites_random_human,1,0 days 00:00:05.000000000,5,0.825,5.976417e-232,0.085570
4,4,A1U9XC376J6ZQ2,"of course , i have also seen brilliant women w...",":) anyways , you ask what love is blind really...",3,raw_data/gyafc_rewrites_random_human,1,0 days 00:00:10.000000000,10,0.825,5.976417e-232,0.085570
